In [1]:
from scipy.io import loadmat
import numpy as np
train1 = loadmat('dataset/scene_train.mat')
train_x1 = train1.get('X')
train_y1 = train1.get('Y')

test1 = loadmat('dataset/scene_test.mat')
test_x1 = test1.get('X')
test_y1 = test1.get('Y')

x1 = np.asmatrix(train_x1).astype(np.double)
y1 = train_y1.astype(np.double)

<div style="direction:rtl;line-height:300%;"><font face="XB Zar" size=5>
با انتخاب C1 به عنوان C ،C برای هر دسته متناسب با تعدا نمونه های آن دسته است و با C2 برای هر دو دسته مساوی ۱ است.
</font>
</div>

In [11]:
def calcualte_c_1(x,y):
    N = len(x)
    c1_index = np.array([i for i in range(len(y[0])) if y[0,i] == 1])
    c1 = len(c1_index)/N
    c2 = 1- c1
    C1 = np.ones(N) * c2
    for i in c1_index:
        C1[i] = c2
    return C1
def calculate_c_2(x):
    N = len(x)
    C2 = np.ones(N)
    return C2


<div style="direction:rtl;line-height:300%;"><font face="XB Zar" size=5>
با استفاده از کتابخانه cvxopt مساله quadratic programming را حل کردیم. 
</br>
البته ابتدا به فرم متناسب این کتابخانه در آوردیم سپس حل کننده را روی این ماتریس ها صدا زدیم.
همانطور که دیده میشود alfa هیچ گاه صفر نشده ولی عددی بسیار پایین میشود لذا alfa کوچکتر از مثلا e^-5 را صفر درنظر می‌گیریم.
</font>
</div>

In [3]:
import numpy as np
from cvxopt import matrix, solvers
def cvxopt_function(K, y,C):
    N = y.shape[1]
    P = matrix(np.multiply(np.dot(y.T,y),K))
    P_ = np.dot(np.dot(y.T,y),K)
    q = matrix(-1 * np.ones((N,1)))
    A = matrix(np.matrix(y))
    b = matrix(0.0)
    G = matrix(np.vstack((np.diag(np.ones(N) * -1), np.identity(N))))
    h = matrix(np.hstack((np.zeros(N), C)))
    return solvers.qp(P, q, G, h, A, b)['x']


In [4]:
#finding one support vector
import math
def find_one_sv(alfas):
    for i in range(len(alfas)):
        if alfas[i] >= math.e**(-5):
            return i


In [5]:
#calculate w
def calculate_w(alfas,x,y):
    w = 0
    for i in range(len(alfas)):
        if alfas[i] >= math.e**(-5):
            w += alfas[i]*y[0,i]*x[i]
    return w

In [6]:
#calculate w0
def calculate_w0(sv_index, x, y):
     return y[0,sv_index] - np.dot(w,x[sv_index].T)

In [7]:
def y_(x, w, w0):
    if w0 + np.dot(w,x.T) >= 0:
        return +1
    return -1

In [8]:
def calculate_precision(test_x,test_y,w,w0):
    N_ = len(test_x)
    positive = 0
    negetive = 0
    positive_correct = 0
    negetive_correct = 0
    for i in range(N_):
        if y_(test_x[i],w,w0) > 0:
            positive += 1
            if test_y[0,i] > 0:
                positive_correct+=1
        else:
            negetive += 1
            if test_y[0,i] < 0:
                negetive_correct+=1
    if positive == 0:
        positive = 1
    if negetive == 0:
        negetive = 1
    print('+1: ', positive_correct/positive)
    print('-1: ', negetive_correct/negetive)

<div style="direction:rtl;line-height:300%;"><font face="XB Zar" size=5>
دقت بر روی داده های تست در دو خانه مقابل نشان داده شده است. اولی برای C1 و دومی برای C2
</br>
</font>
</div>

In [13]:
C = calcualte_c_1(x1,y1)
k = np.dot(x1,x1.T)
result = cvxopt_function(k,y1,C)
print('alfas: ')
print(result)
sv = find_one_sv(result)
w = calculate_w(result, x1, y1)
w0 = calculate_w0(sv, x1, y1)
calculate_precision(test_x1,test_y1,w,w0)

     pcost       dcost       gap    pres   dres
 0: -3.4773e+02 -3.3322e+03  2e+04  3e+00  5e-13
 1: -2.1904e+02 -1.9594e+03  3e+03  3e-01  4e-13
 2: -1.8933e+02 -7.2542e+02  7e+02  6e-02  3e-13
 3: -1.9287e+02 -3.6581e+02  2e+02  1e-02  3e-13
 4: -2.0454e+02 -2.6832e+02  7e+01  3e-03  3e-13
 5: -2.1025e+02 -2.4676e+02  4e+01  1e-03  3e-13
 6: -2.1330e+02 -2.3495e+02  2e+01  3e-04  3e-13
 7: -2.1702e+02 -2.2447e+02  7e+00  4e-05  4e-13
 8: -2.1840e+02 -2.2154e+02  3e+00  5e-06  4e-13
 9: -2.1925e+02 -2.2000e+02  8e-01  4e-15  4e-13
10: -2.1955e+02 -2.1959e+02  4e-02  3e-15  4e-13
11: -2.1956e+02 -2.1956e+02  1e-03  2e-15  4e-13
12: -2.1956e+02 -2.1956e+02  3e-05  1e-15  4e-13
Optimal solution found.
alfas: 
[ 5.55e-09]
[ 8.59e-09]
[ 2.68e-08]
[ 1.53e-08]
[ 9.17e-09]
[ 6.40e-08]
[ 2.59e-08]
[ 1.14e-08]
[ 1.25e-08]
[ 8.40e-09]
[ 1.79e-08]
[ 1.04e-08]
[ 2.16e-07]
[ 9.25e-01]
[ 5.26e-09]
[ 1.16e-08]
[ 9.13e-01]
[ 2.01e-08]
[ 1.10e-08]
[ 7.95e-09]
[ 9.78e-09]
[ 2.63e-08]
[ 7.33e-09]
[ 6.37e

In [15]:
C = calculate_c_2(x1)
k = np.dot(x1,x1.T)
result = cvxopt_function(k,y1,C)
print('alfas: ')
print(result)
sv = find_one_sv(result)
w = calculate_w(result, x1, y1)
w0 = calculate_w0(sv, x1, y1)
calculate_precision(test_x1,test_y1,w,w0)

     pcost       dcost       gap    pres   dres
 0: -3.6090e+02 -3.6103e+03  2e+04  3e+00  4e-13
 1: -2.3391e+02 -2.0645e+03  3e+03  3e-01  4e-13
 2: -2.0295e+02 -7.7093e+02  8e+02  6e-02  3e-13
 3: -2.0708e+02 -3.9279e+02  2e+02  1e-02  3e-13
 4: -2.1961e+02 -2.9187e+02  8e+01  3e-03  3e-13
 5: -2.2633e+02 -2.6562e+02  4e+01  1e-03  3e-13
 6: -2.2900e+02 -2.5346e+02  2e+01  2e-04  4e-13
 7: -2.3269e+02 -2.4312e+02  1e+01  4e-15  4e-13
 8: -2.3511e+02 -2.3839e+02  3e+00  2e-14  4e-13
 9: -2.3590e+02 -2.3698e+02  1e+00  7e-15  4e-13
10: -2.3626e+02 -2.3644e+02  2e-01  7e-15  4e-13
11: -2.3633e+02 -2.3634e+02  6e-03  2e-14  4e-13
12: -2.3634e+02 -2.3634e+02  1e-04  2e-14  4e-13
Optimal solution found.
alfas: 
[ 2.05e-08]
[ 3.26e-08]
[ 9.10e-08]
[ 5.76e-08]
[ 3.54e-08]
[ 2.50e-07]
[ 1.03e-07]
[ 4.92e-08]
[ 4.73e-08]
[ 3.14e-08]
[ 7.66e-08]
[ 3.71e-08]
[ 5.04e-07]
[ 1.00e+00]
[ 1.98e-08]
[ 4.50e-08]
[ 9.83e-01]
[ 7.55e-08]
[ 4.06e-08]
[ 2.92e-08]
[ 3.54e-08]
[ 1.03e-07]
[ 2.74e-08]
[ 6.60e

<div style="direction:rtl;line-height:300%;"><font face="XB Zar" size=5>
<font color=red size=7>
<p></p>
<div align=center>ج </div>
</font>
</div>

In [16]:
from scipy.io import loadmat
train2 = loadmat('dataset/Mnist_train.mat')
train_x2 = train2.get('X')
train_y2 = train2.get('Y')

test2 = loadmat('dataset/Mnist_test.mat')
test_x2 = test2.get('X')
test_y2 = test2.get('Y')

x2 = np.asmatrix(train_x2).astype(np.double)
y2 = train_y2.astype(np.double)

In [17]:
import math
def RBF(x1, x2):
    return math.exp(-0.0006*np.linalg.norm(x1-x2)**2)
def RBF_kernel(x):
    N = len(x)
    kernel = np.zeros((N, N))
    for i in range(N):
        for j in range(N):
            kernel[i,j] = RBF(x[i], x[j])
    return kernel


In [18]:
def calculate_accuracy(test_x,test_y,w,w0):
    N_ = len(test_x)
    positive = 0
    negetive = 0
    positive_correct = 0
    negetive_correct = 0
    for i in range(N_):
        if y_(test_x[i],w,w0) > 0:
            positive += 1
            if test_y[0,i] > 0:
                positive_correct+=1
        else:
            negetive += 1
            if test_y[0,i] < 0:
                negetive_correct+=1
    print('accuracy: ', (negetive_correct+positive_correct)/(negetive +positive))

<div style="direction:rtl;line-height:300%;"><font face="XB Zar" size=5>
 در خانه مقابل دقت بدون استفاده از کرنل در خروجی نشان داده شده است.
</br>
</font>
</div>

In [19]:
C = calculate_c_2(x2)
k = np.dot(x2,x2.T)
result = cvxopt_function(k,y2,C)
print('alfas: ')
print(result)
sv = find_one_sv(result)
w = calculate_w(result, x2, y2)
w0 = calculate_w0(sv, x2, y2)
calculate_accuracy(test_x2,test_y2,w,w0)

     pcost       dcost       gap    pres   dres
 0: -2.6469e+02 -2.8963e+03  2e+04  3e+00  5e-10
 1: -1.7634e+02 -1.7732e+03  3e+03  4e-01  5e-10
 2: -1.3725e+02 -6.7004e+02  9e+02  1e-01  2e-10
 3: -1.2283e+02 -2.4748e+02  2e+02  2e-02  7e-11
 4: -1.2348e+02 -1.7648e+02  9e+01  9e-03  3e-11
 5: -1.2424e+02 -1.3829e+02  2e+01  2e-03  1e-11
 6: -1.2494e+02 -1.2604e+02  2e+00  1e-04  1e-12
 7: -1.2498e+02 -1.2524e+02  4e-01  3e-05  3e-13
 8: -1.2499e+02 -1.2509e+02  1e-01  8e-06  9e-14
 9: -1.2500e+02 -1.2504e+02  5e-02  3e-06  3e-14
10: -1.2500e+02 -1.2501e+02  1e-02  5e-07  8e-15
11: -1.2500e+02 -1.2500e+02  6e-04  3e-08  1e-14
12: -1.2500e+02 -1.2500e+02  6e-06  3e-10  1e-14
Optimal solution found.
alfas: 
[-5.74e-11]
[-1.02e-10]
[ 3.24e-09]
[-1.16e-11]
[-8.63e-11]
[ 4.99e-10]
[-1.16e-10]
[-1.07e-10]
[-7.64e-11]
[-1.25e-10]
[ 1.00e+00]
[ 5.81e-11]
[-1.27e-10]
[-7.62e-11]
[-7.51e-11]
[ 3.39e-09]
[-1.22e-10]
[-5.43e-11]
[ 6.97e-10]
[-1.24e-10]
[-1.15e-10]
[-1.19e-10]
[-1.04e-10]
[ 3.51e

<div style="direction:rtl;line-height:300%;"><font face="XB Zar" size=5>
 در خانه مقابل دقت با استفاده کرنل RBF در خروجی نشان داده شده است.
</br>
</font>
</div>

In [20]:
C = calculate_c_2(x2)
k = RBF_kernel(x2)
result = cvxopt_function(k,y2,C)
print('alfas: ')
print(result)
sv = find_one_sv(result)
w = calculate_w(result, x2, y2)
w0 = calculate_w0(sv, x2, y2)
calculate_accuracy(test_x2,test_y2,w,w0)

     pcost       dcost       gap    pres   dres
 0: -1.4387e+02 -1.9707e+03  9e+03  2e+00  2e-14
 1: -1.1581e+02 -1.1229e+03  2e+03  4e-01  2e-14
 2: -1.1263e+02 -5.4133e+02  7e+02  1e-01  1e-14
 3: -1.2179e+02 -2.5096e+02  2e+02  3e-02  8e-15
 4: -1.2492e+02 -1.7156e+02  7e+01  8e-03  5e-15
 5: -1.2562e+02 -1.4100e+02  2e+01  3e-03  4e-15
 6: -1.2533e+02 -1.2857e+02  5e+00  5e-04  6e-15
 7: -1.2503e+02 -1.2520e+02  3e-01  3e-05  9e-15
 8: -1.2500e+02 -1.2500e+02  3e-03  3e-07  1e-14
 9: -1.2500e+02 -1.2500e+02  3e-05  3e-09  1e-14
Optimal solution found.
alfas: 
[-9.95e-10]
[-1.19e-09]
[ 3.27e-08]
[-5.73e-10]
[-1.14e-09]
[ 2.57e-09]
[-1.37e-09]
[-1.19e-09]
[-1.05e-09]
[-1.40e-09]
[ 1.00e+00]
[-1.12e-10]
[-1.41e-09]
[-1.14e-09]
[-1.13e-09]
[ 6.75e-08]
[-1.44e-09]
[-1.20e-09]
[ 3.43e-09]
[-1.39e-09]
[-1.29e-09]
[-1.39e-09]
[-1.31e-09]
[-5.53e-10]
[-1.27e-09]
[-8.81e-10]
[-9.41e-10]
[-1.34e-09]
[-1.18e-09]
[-1.22e-09]
[-1.09e-09]
[-1.36e-09]
[ 1.03e-08]
[-3.19e-10]
[-1.01e-09]
[-1.27e-09